# Generating a Nominal Trajectory

This notebook demonstrates how to generate a nominal trajectory in the North-East-Down (NED) local coordinate system and convert it to WGS84 geographic coordinates (longitude, latitude, and height). The trajectory is based on a defined starting point and direction, with the ability to save the resulting positions in a CSV file for further analysis or visualization.

## Objective

- Define the starting point in geographic coordinates (latitude, longitude, height).
- Specify a route direction (azimuth) and slope (pitch) to generate a trajectory.
- Compute a series of local NED coordinates based on the specified direction and slope.
- Convert the local NED coordinates to WGS84 geographic coordinates (latitude, longitude, height).
- Save the resulting trajectory as a CSV file with appropriate headers.

## Step 1: Import Required Libraries

We begin by importing necessary libraries: 

In [39]:
from sargeom.coordinates import Cartographic, CartesianLocalNED
from scipy.spatial.transform import Rotation
import numpy as np

## Step 2: Define the NED Frame Origin

The origin of the NED coordinate system is defined in terms of latitude, longitude, and height. This point serves as the reference for all local NED coordinates. In this example, we use the following coordinates:

- Latitude: 3.2431077847784763490°
- Longitude: 42.475181371623002°
- Height: 3818 meters

In [40]:
ned_frame_origin = Cartographic(
    longitude=3.2431077847784763490, # degrees
    latitude=42.475181371623002, # degrees
    height=3818.0 # meters
)

## Step 3: Specify the Route and Slope

The direction of travel (route) and the slope (pitch) of the trajectory are defined in degrees. In this example:

- Route (azimuth): 270.615° (heading direction)
- Slope (pitch): -0.087° (downward slope)
- Step size: 0.28 meters between each sample
- Number of trajectory samples: 100
- First sample distance from the origin: 10 meters (this will be the distance of the first trajectory point from the NED origin)

In [41]:
heading_angle = 270.615 # degrees
pitch_angle = -0.087 # degrees
sample_step = 0.28 # meters
number_of_samples = 30547
initial_distance_from_origin = 3170 # meters

## Step 4: Compute the Direction of Travel

We compute the direction of the trajectory in the NED (North-East-Down) coordinate system using a rotation matrix. The `scipy.spatial.transform.Rotation` class allows us to define rotations based on Euler angles. Here we use the "ZYX" intrinsic rotation convention to apply the azimuth (route) and pitch (slope).

In [42]:
travel_direction = Rotation.from_euler(
    "ZYX",  # Intrinsic rotations
    [heading_angle, pitch_angle, 0], # Route and slope angles (yaw, pitch, roll)
    degrees=True,
).apply([1, 0, 0]) # Apply the rotation to the x-axis direction

## Step 5: Define the Trajectory

The trajectory is created by generating a sequence of NED coordinates. We define the displacement along the trajectory by multiplying the direction vector by a step size and creating an array of sample points along the trajectory.

In [43]:
distance_samples = initial_distance_from_origin + np.arange(0, number_of_samples) * sample_step # Create array of distances

# Generate the trajectory in NED coordinates
trajectory_ned = CartesianLocalNED.from_array(np.outer(distance_samples, travel_direction), origin=ned_frame_origin)

## Step 6: Convert NED Coordinates to WGS84

Once the trajectory is defined in NED coordinates, we convert it to WGS84 geographic coordinates (latitude, longitude, and height) using the .to_ecef().to_cartographic() method chain. This will allow us to obtain the desired geographic positions in degrees and meters.

In [44]:
trajectory_wgs84 = trajectory_ned.to_ecef().to_cartographic()

## Step 7: Save the Trajectory to a CSV File

Finally, we save the generated trajectory to a CSV file. The CSV file includes headers with descriptions of the coordinate fields, making it easy to understand the data. The file will store the coordinates with appropriate precision.

In [45]:
output_filename = "nominal_trajectory.csv"
CSV_HEADER_DESCRIPTION = """# Fields descriptions:
# -------------------
#    o Positions as WGS84 Geographic coordinates:
#        - LON_WGS84_DEG [°]: The longitude coordinate
#        - LAT_WGS84_DEG [°]: The latitude coordinate
#        - HEIGHT_WGS84_M [m]: The ellipsoïdal height coordinate

LON_WGS84_DEG;LAT_WGS84_DEG;HEIGHT_WGS84_M"""

np.savetxt(
    output_filename, trajectory_wgs84.to_array(),
    fmt=['%.12f', '%.12f', '%.6f'],
    delimiter=';', newline='\n', comments='',
    encoding='utf8',
    header=CSV_HEADER_DESCRIPTION
)

## Conclusion

In this notebook, we have demonstrated how to generate a nominal trajectory based on a given route and slope, starting from a specified geographic origin. We have also shown how to convert the local NED coordinates to the WGS84 geographic system and save the result in a CSV file. This process can be extended to more complex trajectory modeling or to visualize the trajectory in 3D space.

## Further Reading

- [SciPy Spatial Transformations](https://docs.scipy.org/doc/scipy/reference/spatial.transform.html#module-scipy.spatial.transform) for understanding rotation matrices and Euler angles.